[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io)

# Deep Learning Methods

## Deep Learning - Auto Encoder - Denoising

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 19/01/2026 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0099DeepLearningObjectDetection.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning

# Deep Learning
import torch
import torch.nn            as nn
import torch.nn.functional as F

from torchvision.transforms import v2 as TorchVisionTrns

import torchinfo
import torchvista

from torchmetrics.functional import r2_score

# Miscellaneous
import os
from platform import python_version
import random

# Typing
from typing import Callable, Literal, Optional, Self, Tuple, Union
from numpy.typing import NDArray
from torch import Tensor

# Visualization
import matplotlib.pyplot as plt

# Jupyter
from IPython import get_ipython

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

```python
valToFill = ???
```

 - Multi Line to Fill (At least one)

```python
# You need to start writing
?????
```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

?????
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

# Improve performance by benchmarking
torch.backends.cudnn.benchmark = True

# Reproducibility (Per PyTorch Version on the same device)
# torch.manual_seed(seedNum)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark     = False #<! Makes things slower

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

D_CLASSES   = {ii: str(ii) for ii in range(10)}
L_CLASSES   = [str(ii) for ii in range(10)]
TU_IMG_SIZE = (28, 28, 1)

PROJECT_NAME     = 'FixelCourses'
DATA_FOLDER_NAME = 'DataSets'
BASE_FOLDER_PATH = os.getcwd()[:(len(os.getcwd()) - (os.getcwd()[::-1].lower().find(PROJECT_NAME.lower()[::-1])))]
DATA_FOLDER_PATH = os.path.join(BASE_FOLDER_PATH, DATA_FOLDER_NAME)

TENSOR_BOARD_BASE = 'TB'

In [ ]:
# Download Auxiliary Modules for Google Colab
if runInGoogleColab:
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DataManipulation.py
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DataVisualization.py
    !wget https://raw.githubusercontent.com/FixelAlgorithmsTeam/FixelCourses/master/AIProgram/2024_02/DeepLearningPyTorch.py

In [ ]:
# Courses Packages

from DataManipulation import DownloadUrl
from DataVisualization import AnnotateImage, PlotLabelsHistogram, PlotMnistImages, PlotScatterData
from DeepLearningPyTorch import TrainModel

In [ ]:
# General Auxiliary Functions

def TensorImageNumpy( tZ: Tensor ) -> NDArray:
    """
    Converts a PyTorch Tensor to a Numpy Array.
    """
    mZ = tZ.squeeze()
    mX = mZ.detach().cpu().numpy()

    return mX

class MNISTDatasetCSV(torch.utils.data.Dataset):
    """
    MNIST Dataset from CSV File.  
    Supports Image Classification and Self Supervised Learning tasks.
    """

    def __init__( self: Self, csvFilePath: str, subSetType: Literal['All', 'Train', 'Val'], *, tuImgSize: Tuple[int, ...] = (28, 28), hImgTrns: Optional[Callable] = None, hFeatTrans: Optional[Callable] = None, hTgtTrns: Optional[Callable] = None ) -> None:
        """
        Constructor Method.

        Parameters
        ----------
        csvFilePath : str
            Path / URL to the CSV file.
        subSetType : Literal['All', 'Train', 'Val']
            Subset type: 'All' for the entire dataset, 'Train' for training set, 'Val' for validation set
        tuImgSize : Tuple[int, ...], optional
            Image size tuple, by default (28, 28)
        imgTrns : Optional[Callable], optional
            Transform to be applied to the images, by default None
        """

        dfData = pd.read_csv(csvFilePath)
        match subSetType:
            case 'All':
                pass
            case 'Train':
                dfData = dfData.iloc[:60000, :].reset_index(drop = True)
            case 'Val':
                dfData = dfData.iloc[60000:, :].reset_index(drop = True)
            case _:
                raise ValueError(f'Unsupported subset type: {subSetType}')
        
        dsLbl = dfData.iloc[:, -1]
        dsLbl = dsLbl.astype(np.uint8)

        self._dfData    = dfData
        self._dsLbl     = dsLbl
        self._tuImgSize = tuImgSize
        self._hImgTrns  = hImgTrns
        self._hTgtTrns  = hTgtTrns
        self._hFeatTrns = hFeatTrans

    def __len__( self: Self ) -> int:
        """
        Returns the number of samples in the dataset.
        """
        return len(self._dfData)

    def __getitem__( self: Self, idx: int ) -> Tuple[Tensor, Union[int, Tensor]]:
        """
        Returns the sample at the given index.

        Parameters
        ----------
        idx : int
            Index of the sample to be retrieved.

        Returns
        -------
        Tuple[torch.Tensor, int]
            Tuple containing the image tensor and its label.
        """
        
        dsRow = self._dfData.iloc[idx]
        tX    = dsRow.iloc[:-1].to_numpy(np.uint8, copy = True)
        tX    = np.reshape(tX, self._tuImgSize)
        valY  = dsRow.iloc[-1]

        if self._hImgTrns:
            # Assuming Torchvision v2 transforms
            tX = self._hImgTrns(tX)

        # Create a copy for feature transform
        # Handle the case of NumPy Array and Torch Tensor
        if isinstance(tX, np.ndarray):
            tY = tX.copy()
        else:
            tY = tX.clone()
        
        if self._hFeatTrns:
            tX = self._hFeatTrns(tX)
        
        if self._hTgtTrns:
            tY = self._hTgtTrns(tY)

        return tX, (valY, tY)
    
    def GetLabels( self: Self ) -> NDArray:
        """
        Returns all labels in the dataset.

        Returns
        -------
        NDArray
            Array of labels.
        """
        return self._dsLbl.to_numpy()
    
    def SetTransform( self: Self, trnsType: Literal['Feature', 'Image', 'Target'], hTrns: Optional[Callable] ) -> None:
        """
        Sets the image transform for the dataset.

        Parameters
        ----------
        imgTrns : Optional[Callable]
            Transform to be applied to the images
        """

        match trnsType:
            case 'Feature':
                self._hFeatTrns = hTrns
            case 'Image':
                self._hImgTrns = hTrns
            case 'Target':
                self._hTgtTrns = hTrns
            case _:
                raise ValueError(f'Unsupported transform type: {trnsType}')

* <font color='blue'>(**!**)</font> Go through `MNISTDatasetCSV` class. Understand how it serves the _Self Supervised_ concept.

## Auto Encoder

![](https://i.imgur.com/njKqIRZ.png)
<!-- ![](https://i.postimg.cc/sX3SJmTM/Diagrams-Auto-Encoder.png) -->

An _Auto Encoder_ is a model used to learn efficient embeddings of unlabeled data utilizing _Self Supervised_ learning.  
The model is composed of:
 - _Encoder_: Transform the input $\color{cyan}{\boldsymbol{x}}$ into an embedding $\color{green}{\boldsymbol{z}}$ in the _latent space_.  
 - _Decoder_: Reconstruct from the embedding $\color{green}{\boldsymbol{z}}$ a representation which is a function of the input.

Some use cases of _Auto Encoders_:

 - Low Dimension Representation  
   In this case the output should match the input and $\color{green}{\boldsymbol{z}}$ is the low dimension embedding.
 - **Denoising**   
   In this case the input is a noisy sample and the reconstruction is trying to match a noiseless sample.
 - Outlier Detection  
   In this case the input should match the output. The decision function is based on the reconstruction error.
 - Data Generation  
   In this case the input should match the output.

* <font color='brown'>(**#**)</font> Data Generation models require some regularization of latent space to make it suitable for sampling.
* <font color='brown'>(**#**)</font> The **Denoising** task is an example for reversing any operation which its forward step can applied.

In the context of image input the _Encoder_ and _Decoder_ are composed of Spatial specialized components such as CNN's.

This notebook demonstrates:
 - Building a Dataset for _Self Supervised_ context.  
   The dataset will employ a transform of the features before entering the model.
 - Building an _Encoder_ based on CNN.
 - Building a _Decoder_ based on CNN.  
 - Building a Multi Head Model  
   The _Auto Encoder_ output will be composed of a reconstruction and classification heads.
 - Training the _Auto Encoder_ using a self supervised manner.  
   In this case the motivation is to reverse (Denoising) an operation (Adding noise) on the input features.
 - Analysis of the geometry of the latent space.

</br>

* <font color='brown'>(**#**)</font> Some Encoder Decoders models are based on the _Vision Transformer_ (ViT).

In [ ]:
# Parameters

# Data
csvFileName = 'MNIST.csv'
csvFileUrl  = r'https://huggingface.co/datasets/Royi/MNIST/resolve/main/MNIST.csv'

numCls          = len(L_CLASSES) #<! Number of classes

# Model
latDim     = 32

# Training
batchSize   = 512
numWorkers  = 2 #<! Number of workers
numEpochs   = 45

# Visualization
numImg = 3

## Generate / Load Data

The data is the MNIST Dataset.  
This section:

 - Defines the `Dataset` class.  
   It should support the case the labels are the image itself.
 - Create a _Train_ and _Validation_ datasets.
 - Plot the data.
 - Define the _Augmentation_ / _Transform_.
 - Define the `Dataloader`.

In [ ]:
# Download Data (CSV)

csvFilePath = os.path.join(DATA_FOLDER_PATH, csvFileName)
csvFilePath = DownloadUrl(csvFileUrl, csvFilePath)

In [ ]:
# Data Set

dsTrain = MNISTDatasetCSV(csvFilePath, 'Train')
dsVal   = MNISTDatasetCSV(csvFilePath, 'Val')

print(f'The number of samples in training data set  : {len(dsTrain)}')
print(f'The number of samples in validation data set: {len(dsVal)}')

In [ ]:
# Element of the Data Set / Data Sample

tX, tuY = dsTrain[0]
valY    = tuY[0]
tY      = tuY[1]

print(f'The features shape: {tX.shape}')
print(f'The target shape  : {tY.shape}')
print(f'The label         : {valY}')

### Plot the Data

In [ ]:
# Plot the Data

mX = np.zeros((9, 28 * 28), dtype = np.uint8)
vY = np.zeros((9,), dtype = np.uint8)

for ii in range(9):
    randIdx = random.randint(0, len(dsTrain) - 1)
    tX, tuY = dsTrain[randIdx]
    valY    = tuY[0]
    mX[ii]  = tX.flatten()
    vY[ii]  = valY

hF = PlotMnistImages(mX, vY, 3, 3)

In [ ]:
# Plot Single Sample

randIdx = random.randint(0, len(dsTrain) - 1)
tX, tuY = dsTrain[randIdx]
valY    = tuY[0]

hF, hA = plt.subplots(figsize = (7, 7))
hA.imshow(tX, cmap = 'gray')
hA.set_title(f'Sample Index: {randIdx}, Label: {valY}')
AnnotateImage(tX, hA, fontSize = 6);

In [ ]:
# Histogram of Labels

hF, vHa = plt.subplots(nrows = 1, ncols = 2, figsize = (8, 4))
vHa = vHa.flat

hA = PlotLabelsHistogram(dsTrain.GetLabels(), hA = vHa[0], lClass = L_CLASSES)
hA.set_title('Histogram of Labels, Training Set');

hA = PlotLabelsHistogram(dsVal.GetLabels(), hA = vHa[1], lClass = L_CLASSES)
hA.set_title('Histogram of Labels, Validation Set');

### Augmentation / Transform

In [ ]:
# Loader Transform

oTrnsTrain = TorchVisionTrns.Compose([
    TorchVisionTrns.ToImage(),
    TorchVisionTrns.ToDtype(torch.float, scale = True),
    TorchVisionTrns.RandomRotation(degrees = 15), #<! Augmentation by rotation
])

oTrnsVal = TorchVisionTrns.Compose([
    TorchVisionTrns.ToImage(),
    TorchVisionTrns.ToDtype(torch.float, scale = True),
])

oFeatTrns = TorchVisionTrns.Compose([
    TorchVisionTrns.GaussianNoise(mean = 0.0, sigma = 0.1),
])

In [ ]:
# Apply Transforms

dsTrain.SetTransform('Image', oTrnsTrain)
dsTrain.SetTransform('Feature', oFeatTrns)
dsVal.SetTransform('Image', oTrnsVal)

In [ ]:
# Element of the Data Set / Data Sample

tX, tuY = dsTrain[0]
valY    = tuY[0]
tY      = tuY[1]

print(f'The features shape: {tX.shape}')
print(f'The target shape  : {tY.shape}')
print(f'The label         : {valY}')

In [ ]:
# Plot Single Sample

randIdx = random.randint(0, len(dsTrain) - 1)
tX, tuY = dsTrain[randIdx]
valY    = tuY[0]

mX = TensorImageNumpy(tX)

hF, hA = plt.subplots(figsize = (4, 4))
hA.imshow(mX, cmap = 'gray')
hA.set_title(f'Sample Index: {randIdx}, Label: {valY}');

* <font color='red'>(**?**)</font> What other augmentation would work for this case? Should flips be used?

### Data Loaders

In [ ]:
# Data Loader

dlTrain = torch.utils.data.DataLoader(dsTrain, shuffle = True, batch_size = batchSize, num_workers = 0)
dlVal   = torch.utils.data.DataLoader(dsVal, shuffle = False, batch_size = 2 * batchSize, num_workers = 0)

In [ ]:
# Iterate on the Loader
# The first batch.
tX, tuY = next(iter(dlTrain)) #<! PyTorch Tensors

print(f'The batch features dimensions: {tX.shape}')
print(f'The batch labels dimensions: {tuY[0].shape}')
print(f'The batch targets dimensions: {tuY[1].shape}')

## Build AutoEncoder Model

In [ ]:
# Encoder Decoder Model

class AutoEncoder(nn.Module):
    def __init__(self, latDim: int, numCls: int, α: float = 0.1):
        super().__init__()

        # Encoder: 1x28x28 -> 64x7x7 -> 2
        self.oEnc = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding = 1, bias = False),
            nn.BatchNorm2d(16),
            nn.LeakyReLU(α, inplace = True),

            nn.Conv2d(16, 32, 4, stride = 2, padding = 1, bias = False),   #<! 14x14
            nn.BatchNorm2d(32),
            nn.LeakyReLU(α, inplace = True),

            nn.Conv2d(32, 64, 4, stride = 2, padding = 1, bias = False),  #<! 7x7
            nn.BatchNorm2d(64),
            nn.LeakyReLU(α, inplace = True),

            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 256),
            nn.LeakyReLU(α, inplace = True),
            nn.Linear(256, latDim),
        )

        # Decoder: 2 -> 128x7x7 -> 1x28x28
        self.oDec = nn.Sequential(
            nn.Linear(latDim, 256),
            nn.LeakyReLU(α, inplace = True),
            nn.Linear(256, 64 * 7 * 7),
            nn.LeakyReLU(α, inplace = True),
            nn.Unflatten(1, (64, 7, 7)),

            nn.ConvTranspose2d(64, 32, 4, stride = 2, padding = 1, bias = False),  #<! 14x14
            nn.BatchNorm2d(32),
            nn.LeakyReLU(α, inplace = True),

            nn.ConvTranspose2d(32, 16, 4, stride = 2, padding = 1, bias = False),   #<! 28x28
            nn.BatchNorm2d(16),
            nn.LeakyReLU(α, inplace = True),

            nn.Conv2d(16, 1, 3, padding = 1),
            nn.Sigmoid(),  #<! MNIST scaled to [0,1]
        )

        # Classifier
        self.oCls = nn.Sequential(
            nn.Linear(latDim, 64),
            nn.LeakyReLU(α, inplace = True),
            nn.Linear(64, numCls),
        )

    def forward(self, tX: Tensor) -> Tensor:
        
        tZ    = self.oEnc(tX) #<! Latent Space (Embedding vector)
        tY    = self.oCls(tZ) #<! Classifier Output
        tXHat = self.oDec(tZ) #<! Reconstructed Image
        
        return tY, tXHat

In [ ]:
# The Model Object

oModel = AutoEncoder(latDim, numCls)

In [ ]:
# Model Summary

torchinfo.summary(oModel, (batchSize, *(TU_IMG_SIZE[::-1])), col_names = ['kernel_size', 'output_size', 'num_params'], device = 'cpu', row_settings = ['depth', 'var_names'])

In [ ]:
# Model Graph

torchvista.trace_model(oModel, torch.randn(16, *(TU_IMG_SIZE[::-1])))

## Train the Model

This section defines:
 - Loss Class  
   Composed of 2 loses:
   - The Reconstruction Loss  
     Based on the [MSE](https://en.wikipedia.org/wiki/Mean_squared_error) or [MAE](https://en.wikipedia.org/wiki/Mean_absolute_error) loss.  
     Drives reconstruction of the image at output.
   - The Classification Loss
     Based on the [Cross Entropy](https://en.wikipedia.org/wiki/Cross-entropy) Loss.  
     Tries to make the features in the latent space "classification friendly" by a _Linear Classifier_.  
     Namely tries to make them separate.
 - Score  
   The R2 score.

In [ ]:
# Check GPU Availability

runDevice = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') #<! The 1st CUDA device

In [ ]:
# The Loss Class
class AutoEncoderLoss(nn.Module):
    def __init__( self, recLossType: Literal['MAE', 'MSE'], λRec: float, λCls: float ) -> None:
        super().__init__()

        match recLossType:
            case 'MAE':
                self.oRecLoss = nn.L1Loss()
            case 'MSE':
                self.oRecLoss = nn.MSELoss()
            case _:
                raise ValueError(f'Unsupported loss type: {lossType}')
        
        self.oClsLoss = nn.CrossEntropyLoss()
        self.λRec    = λRec
        self.λCls    = λCls
    
    def forward( self, tuYHat: Tuple[Tensor, Tensor], tuY: Tuple[Tensor, Tensor] ) -> Tensor:

        tYHat, tXHat = tuYHat
        tY,    tX    = tuY
        
        recLoss = self.oRecLoss(tXHat, tX)
        clsLoss = self.oClsLoss(tYHat, tY)
        
        return self.λRec * recLoss + self.λCls * clsLoss

* <font color='red'>(**?**)</font> If one wants to optimize the R2 Score, how should the `λRec` and `λCls` be set? Think about the training and validation score.

In [ ]:
# The Score Class
class AutoEncoderScore(nn.Module):
    def __init__( self ) -> None:
        super().__init__()
    
    def forward( self, tuYHat: Tuple[Tensor, Tensor], tuY: Tuple[Tensor, Tensor] ) -> Tensor:

        _, tXHat = tuYHat
        _, tX    = tuY
        
        r2Score = r2_score(tXHat.view(-1), tX.view(-1))
        
        return r2Score

In [ ]:
# Loss and Score
hL = AutoEncoderLoss('MSE', 0.95, 0.05)
hS = AutoEncoderScore()
hL = hL.to(runDevice) #<! Not required!
hS = hS.to(runDevice)

In [ ]:
# Training the Model
oModel = oModel.to(runDevice) #<! Transfer model to device
oOpt = torch.optim.AdamW(oModel.parameters(), lr = 6e-4, betas = (0.9, 0.99), weight_decay = 1e-3) #<! Define optimizer
oSch = torch.optim.lr_scheduler.OneCycleLR(oOpt, max_lr = 2e-2, total_steps = numEpochs)
oRunModel, lTrainLoss, lTrainScore, lValLoss, lValScore, lLearnRate = TrainModel(oModel, dlTrain, dlVal, oOpt, numEpochs, hL, hS, oSch = oSch)

In [ ]:
# Plot Training Phase

hF, vHa = plt.subplots(nrows = 1, ncols = 3, figsize = (15, 5))
vHa = np.ravel(vHa)

hA = vHa[0]
hA.plot(lTrainLoss, lw = 2, label = 'Train')
hA.plot(lValLoss, lw = 2, label = 'Validation')
hA.set_title(f'AutoEncoder Loss')
hA.set_xlabel('Epoch')
hA.set_ylabel('Loss')
hA.legend()

hA = vHa[1]
hA.plot(lTrainScore, lw = 2, label = 'Train')
hA.plot(lValScore, lw = 2, label = 'Validation')
hA.set_title('AutoEncoder Score')
hA.set_xlabel('Epoch')
hA.set_ylabel('Score')
hA.legend()

hA = vHa[2]
hA.plot(lLearnRate, lw = 2)
hA.set_title('Learn Rate Scheduler')
hA.set_xlabel('Epoch')
hA.set_ylabel('Learn Rate');

In [ ]:
# Inference Mode

oModel = oModel.eval()

In [ ]:
# Sample from Train
tX, valY = dsTrain[7]

tX = tX.to(runDevice).unsqueeze(0)

with torch.inference_mode():
    _, tZ = oModel(tX)
mZ = TensorImageNumpy(tZ)
mX = TensorImageNumpy(tX)

hF, vHa = plt.subplots(nrows = 1, ncols = 2, figsize = (6, 3))
vHa = vHa.flat

hA = vHa[0]
hA.imshow(mX, cmap = 'gray');
hA = vHa[1]
hA.imshow(mZ, cmap = 'gray');

In [ ]:
# Sample from Validation
tX, valY = dsVal[23]

tX = tX.to(runDevice).unsqueeze(0)

with torch.inference_mode():
    _, tZ = oModel(tX)
mZ = TensorImageNumpy(tZ)
mX = TensorImageNumpy(tX)

hF, vHa = plt.subplots(nrows = 1, ncols = 2, figsize = (6, 3))
vHa = vHa.flat

hA = vHa[0]
hA.imshow(mX, cmap = 'gray');
hA = vHa[1]
hA.imshow(mZ, cmap = 'gray');

* <font color='blue'>(**!**)</font> Change `λRec` and `λCls` an retrain.
* <font color='green'>(**@**)</font> Create code to evaluate teh classification. Plot the Confusion Matrix.
* <font color='green'>(**@**)</font> Create a loop to optimize the Hyper Parameters of the model. 